In [62]:
import gmaps
import time
import requests
import pandas as pd
import numpy as np
from citipy import citipy
import matplotlib.pyplot as plot
from config import weather_api_key
from config import g_key
from datetime import datetime
from scipy.stats import linregress

In [63]:
city_data_df=pd.read_csv("weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [64]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [65]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [66]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [67]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [68]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [69]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
7,7,Touros,BR,2020-10-21 19:27:37,-5.20,-35.46,77.00,94,75,11.41
12,12,Santa Isabel Do Rio Negro,BR,2020-10-21 19:32:02,-0.41,-65.02,91.85,50,88,3.20
14,14,Orangeburg,US,2020-10-21 19:29:21,33.49,-80.86,81.00,61,1,5.82
19,19,Saint-Joseph,RE,2020-10-21 19:32:07,-21.37,55.62,75.20,69,23,16.11
20,20,Dingle,PH,2020-10-21 19:29:49,11.00,122.67,76.32,91,100,6.82
23,23,Elizabeth City,US,2020-10-21 19:32:09,36.29,-76.25,82.40,61,1,8.05
24,24,Nioro,GM,2020-10-21 19:32:10,13.35,-15.75,82.49,63,77,7.74
26,26,Zaraza,VE,2020-10-21 19:32:11,9.35,-65.32,88.86,70,4,3.38
29,29,Mahibadhoo,MV,2020-10-21 19:32:12,3.78,72.97,81.79,80,2,12.91
30,30,Victoria,HK,2020-10-21 19:27:25,22.29,114.16,75.00,66,100,1.01


In [70]:
preferred_cities_df.count()

City_ID       287
City          287
Country       285
Date          287
Lat           287
Lng           287
Max Temp      287
Humidity      287
Cloudiness    287
Wind Speed    287
dtype: int64

In [71]:
preferred_cities_df=preferred_cities_df.dropna()

In [72]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Touros,BR,77.00,-5.20,-35.46,
12,Santa Isabel Do Rio Negro,BR,91.85,-0.41,-65.02,
14,Orangeburg,US,81.00,33.49,-80.86,
19,Saint-Joseph,RE,75.20,-21.37,55.62,
20,Dingle,PH,76.32,11.00,122.67,
23,Elizabeth City,US,82.40,36.29,-76.25,
24,Nioro,GM,82.49,13.35,-15.75,
26,Zaraza,VE,88.86,9.35,-65.32,
29,Mahibadhoo,MV,81.79,3.78,72.97,
30,Victoria,HK,75.00,22.29,114.16,


In [73]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Touros,BR,77.00,-5.20,-35.46,
12,Santa Isabel Do Rio Negro,BR,91.85,-0.41,-65.02,
14,Orangeburg,US,81.00,33.49,-80.86,
19,Saint-Joseph,RE,75.20,-21.37,55.62,
20,Dingle,PH,76.32,11.00,122.67,
23,Elizabeth City,US,82.40,36.29,-76.25,
24,Nioro,GM,82.49,13.35,-15.75,
26,Zaraza,VE,88.86,9.35,-65.32,
29,Mahibadhoo,MV,81.79,3.78,72.97,
30,Victoria,HK,75.00,22.29,114.16,


In [74]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [75]:
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping")

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not foun

In [77]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Touros,BR,77.00,-5.20,-35.46,INN NEW HORIZON
12,Santa Isabel Do Rio Negro,BR,91.85,-0.41,-65.02,Hotel GSA
14,Orangeburg,US,81.00,33.49,-80.86,Palmetto Inn & Suites
19,Saint-Joseph,RE,75.20,-21.37,55.62,"""Plantation Bed and Breakfast"
20,Dingle,PH,76.32,11.00,122.67,Camp Pasica
23,Elizabeth City,US,82.40,36.29,-76.25,Hampton Inn Elizabeth City
24,Nioro,GM,82.49,13.35,-15.75,
26,Zaraza,VE,88.86,9.35,-65.32,Hotel Zaraza
29,Mahibadhoo,MV,81.79,3.78,72.97,Omadhoo Inn
30,Victoria,HK,75.00,22.29,114.16,Mini Hotel Central


In [78]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [79]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [80]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [81]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [82]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [83]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))